# Website Scraping
### Use this Notebook to scrape the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

### Read data from wikipedia

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #Create a handle, page, to handle the contents of the website
page = requests.get(url) #Store the contents of the website under doc
doc = lh.fromstring(page.content) #Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

#### Get the Columns for the new table

In [4]:
tr_elements = doc.xpath('//tr') #Create empty list
col = []
i = 0 #For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+= 1
    name = t.text_content().rstrip("\n") # ==> USE RSTRIP to REMOVE TRAILING NEW-LINE CHARACTERS
    print("%d: %s" %(i,name))
    col.append((name,[]))

1: Postal Code
2: Borough
3: Neighbourhood


##### Retrieve each row from the web page

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i = 0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().rstrip("\n") # ==> USE RSTRIP to REMOVE TRAILING NEW-LINE CHARACTERS
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
# Ensure we retrieve an equal number of rows
[len(C) for (title,C) in col]

[181, 181, 181]

In [7]:
# Create the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

#### Delete rows that don't have an assigned borough. 

In [8]:
df = df[df.Borough != 'Not assigned']

In [9]:
df.shape

(104, 3)

In [10]:
df.tail(5)

,Postal Code,Borough,Neighbourhood
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
180,,Canadian postal codes,


In [11]:
# If a cell has a borough and there is no assigned neighborhood, then the neighborhood will be the same as the borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

In [12]:
# Delete last row from the table
df = df.iloc[:-1]

In [13]:
df.tail(5)

,Postal Code,Borough,Neighbourhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


# Using Geocoder

### Install Geocoder package

In [14]:
# Geocoder is a simple and consistent geocoding library written in Python.
# See also https://geocoder.readthedocs.io/
# Install Geocoder package
!pip install geocoder

### Call Geocoder API on Open Street Map to retrieve the GPS locations

In [15]:
import geocoder

latitude = []                        # create empty list to temporarily store GPS coordinates
longitude = []                       # create second empty list to store GPS coordinates
lst = df["Neighbourhood"].to_numpy() # retrieve a list of neighbourhoods
    
x = 0
 
# Iterating using while loop 
while x < len(lst): 
    neighbourhood = lst[x]
    query = neighbourhood.split(",", 5)                                                # split variable if multiple neighborhouds exists
    g = geocoder.osm('{}, Toronto, Ontario'.format(query[0]))                          # select the first value from the string, as query[0]
    y = g.lat
    z = g.lng
    
    if len(query) > 1 and not y :                                  # If no GPS was found, we can try to check the next Neighbourhood (if available) 
        g = geocoder.osm('{}, Toronto, Ontario'.format(query[1]))  # Take second variable if g return is empty
        y = g.lat
        z = g.lng  
    
    latitude.append(y)
    longitude.append(z) 
    x = x+1

### Add the results as "latitude" and "longitude" columns to the Table

In [16]:
# Add the two lists as columns to the table
df["Latitude"] = latitude
df["Longitude"] = longitude

### Indentify remaining misfits that have no GPS coordinates

In [17]:
# Indentify misfits that have no GPS coordinates
df1 = df[df.isna().any(axis=1)]
df1.head(20)
#df.loc[60:100,:]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
32,M6E,York,Caledonia-Fairbanks,NaN,NaN
40,M5G,Downtown Toronto,Central Bay Street,NaN,NaN
114,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN
148,M5W,Downtown Toronto,Stn A PO Boxes,NaN,NaN
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",NaN,NaN


### Loop through Borough's to retrieve remaining coordinates

For some neighbourhoods no matching GPS coordinates were found on Open Street Maps. So switching to Borough's instead. 

In [18]:
# loop to retrieve Burough's from Open Street Maps
latitude = []                        
longitude = [] 
lst = df1["Borough"].to_numpy()

x = 0

while x < len(df1):
    borough = lst[x]
    g = geocoder.osm('{}, Toronto, Ontario'.format(borough))  
    a = g.lat
    b = g.lng
    df1['Latitude'].iloc[x] = a
    df1['Longitude'].iloc[x] = b
    x = x+1

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
df1

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
32,M6E,York,Caledonia-Fairbanks,43.689619,-79.479188
40,M5G,Downtown Toronto,Central Bay Street,43.656322,-79.380916
114,M7R,Mississauga,Canada Post Gateway Processing Centre,43.668384,-79.587058
148,M5W,Downtown Toronto,Stn A PO Boxes,43.656322,-79.380916
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.721789,-79.374027


### Update dataframe with remaining coordinates

In [20]:
# Merge results into original table
df.update(df1)

In [21]:
# See if there are still records with no GPS data
print(df[df.isna().any(axis=1)])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood, Latitude, Longitude]
Index: []


In [22]:
#df.head(20)
#df.loc[30:50,:]

### Display final table with geographical coordinates of the neighborhoods of TORONTO

This table was created by scraping Wikipedia and using Geocoder to retrieve the latitude and the longitude coordinates of each neighborhood. 

In [23]:
df.shape

(103, 5)

In [24]:
df.head(30)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.761124,-79.324059
3,M4A,North York,Victoria Village,43.732658,-79.311189
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.620087,-79.512783
9,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701
11,M3B,North York,Don Mills,43.775347,-79.345944
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.653482,-79.383935
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.656500,-79.377114
